In [1]:
from __future__ import absolute_import
from __future__ import print_function
from future.standard_library import install_aliases
install_aliases()

import os
import gzip
import struct
import array
import numpy as np
from urllib.request import urlretrieve

def download(url, filename):
    if not os.path.exists('data'):
        os.makedirs('data')
    out_file = os.path.join('data', filename)
    if not os.path.isfile(out_file):
        urlretrieve(url, out_file)

def mnist():
    base_url = 'http://yann.lecun.com/exdb/mnist/'

    def parse_labels(filename):
        with gzip.open(filename, 'rb') as fh:
            magic, num_data = struct.unpack(">II", fh.read(8))
            return np.array(array.array("B", fh.read()), dtype=np.uint8)

    def parse_images(filename):
        with gzip.open(filename, 'rb') as fh:
            magic, num_data, rows, cols = struct.unpack(">IIII", fh.read(16))
            return np.array(array.array("B", fh.read()), dtype=np.uint8).reshape(num_data, rows, cols)

    for filename in ['train-images-idx3-ubyte.gz',
                     'train-labels-idx1-ubyte.gz',
                     't10k-images-idx3-ubyte.gz',
                     't10k-labels-idx1-ubyte.gz']:
        download(base_url + filename, filename)

    train_images = parse_images('data/train-images-idx3-ubyte.gz')
    train_labels = parse_labels('data/train-labels-idx1-ubyte.gz')
    test_images  = parse_images('data/t10k-images-idx3-ubyte.gz')
    test_labels  = parse_labels('data/t10k-labels-idx1-ubyte.gz')

    return train_images, train_labels, test_images, test_labels

def load_mnist():
    partial_flatten = lambda x : np.reshape(x, (x.shape[0], np.prod(x.shape[1:])))
    one_hot = lambda x, k: np.array(x[:,None] == np.arange(k)[None, :], dtype=int)
    train_images, train_labels, test_images, test_labels = mnist()
    train_images = partial_flatten(train_images) / 255.0
    test_images  = partial_flatten(test_images)  / 255.0
    train_labels = one_hot(train_labels, 10)
    test_labels = one_hot(test_labels, 10)

    return train_images, train_labels, test_images, test_labels

def check_my_network(layer_sizes, network):
    correct = True
    if not len(network)+1 == len(layer_sizes):
        correct = False
        print("The network does not have the correct number of layers")
        print("Number of layers in network: ", len(network))
        print("Expected number of layers: ", len(layer_sizes))
    for i in range(len(network)):
        if not network[i][0].shape == (layer_sizes[i], layer_sizes[i+1]):
            correct = False
            print("Layer " + str(i) + " does not have the correct dimension weights")
            print("Layer weight shape: ", str(network[i][0].shape))
            print("Expected shape: ", str((layer_sizes[i], layer_sizes[i+1])))
            print("################################################################")
        if not network[i][1].shape[0] == layer_sizes[i+1]:
            correct = False
            print("Layer " + str(i) + " does not have the correct dimension biases")
            print("Layer bias shape: ", str(network[i][1].shape[0]))
            print("Expected shape: ", str(layer_sizes[i+1]))
            print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
    if correct:
        print("The network is correct")


# Above is from the helpers.py

In [16]:
import random

### Layer of a network:

In [71]:
class init_layer:
    
    def __init__(self, x , y):
        self.weight_matrix = np.random.rand(x,y)
        self.bias_vector = np.random.rand(y)

        

### Me checking that the layer works

In [72]:
newLayer = init_layer(10,5)
print("The weight matrix")
print(newLayer.weight_matrix)
print("The bias vector")
print(newLayer.bias_vector)

The weight matrix
[[0.37670459 0.99041852 0.18238334 0.65914052 0.27866435]
 [0.14289452 0.25776492 0.73836775 0.4149757  0.36325166]
 [0.02286824 0.04402312 0.45992289 0.82314253 0.61387998]
 [0.19844702 0.05338613 0.24069517 0.27291441 0.26552173]
 [0.31059719 0.32799265 0.20801179 0.20443404 0.80673834]
 [0.79437329 0.38098593 0.63923489 0.96200437 0.18790526]
 [0.96397321 0.12990165 0.35482423 0.30247185 0.44710712]
 [0.0777153  0.24222633 0.58179333 0.54672553 0.02821964]
 [0.29681136 0.57674306 0.27052676 0.1348965  0.80823428]
 [0.75056072 0.36656569 0.5427568  0.87659919 0.22296194]]
The bias vector
[0.23177688 0.1930341  0.59487113 0.35289369 0.86365438]


### The network itself

In [137]:
class init_random_network:
    
    def __init__(self, network_info):
        self.layers = []
        for i in range(len(network_info) -1):
            self.layers.append(init_layer(network_info[i],network_info[i+1]))

    def output_at_layer(self, position):
        try:
            return self.layers[position].weight_matrix, self.layers[position].bias_vector
        except:
            if position > len(layers):
                print("out of index")
            else:
                print("some error occured")

The network being created for an example

In [138]:
random_network = init_random_network([784,200,100,10])

Running through the network to show its properties match the desired properties

In [139]:
for i in range(len(random_network.layers)):
    print("this is the network layer's weight matrix " + str(random_network.layers[i].weight_matrix.shape))
    print("This is the network layer's bias vector " + str(random_network.layers[i].bias_vector.shape))
    

this is the network layer's weight matrix (784, 200)
This is the network layer's bias vector (200,)
this is the network layer's weight matrix (200, 100)
This is the network layer's bias vector (100,)
this is the network layer's weight matrix (100, 10)
This is the network layer's bias vector (10,)
None


In [155]:
data = load_mnist()
#Data takes the form of a tuple
#where the touple is the  (data,labels)
trainingData = np.array([data[0][0],data[0][1]])
trainingLabels = np.array([data[1][0],data[1][1]])
testData = np.array([data[2][0],data[2][1]])
testLabel = np.array([data[3][0],data[3][1]])

# TODO: part d, make forward propagation function